# Finetune Gemma model on LPI(Learning Index Path) Dataset

# Step 1

## Prerequisites

In [ ]:
!nvidia-smi

Fri Oct 18 22:17:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!npm install -g localtunnel
!pip install streamlit
!pip install torch


added 22 packages, and audited 23 packages in 2s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate
!pip3 install -q -U datasets
!pip3 install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 823.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 103.7 MB/s eta 0:00:00


In [ ]:
# prompt: i want to print out the versions only

!nvidia-smi
!echo "bitsandbytes version:"
!pip show bitsandbytes
!echo "peft version:"
!pip show peft
!echo "trl version:"
!pip show trl
!echo "accelerate version:"
!pip show accelerate
!echo "datasets version:"
!pip show datasets
!echo "transformers version:"
!pip show transformers

Fri Oct 18 22:18:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import bitsandbytes as bnb
import transformers
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
from datasets import Dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Now we specify the model ID and then we load it with our previously defined quantization configuration.Now we specify the model ID and then we load it with our previously defined quantization configuration.

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Step 2 - Model loading
We'll load the model using QLoRA quantization to reduce the usage of memory


In [ ]:
model_id = "google/gemma-2-9b"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  {query}
  <end_of_turn>\n<start_of_turn>model


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

In [ ]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Course_Learning_Material"], padding="max_length", truncation=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
result = get_completion(query="code the fibonacci series in python using reccursion", model=model, tokenizer=tokenizer)
print(result)

The 'max_batch_size' argument of HybridCache is deprecated and will be removed in v4.46. Use the more precisely named 'batch_size' argument instead.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



  user
  Below is an instruction that describes a task. Write a response that appropriately completes the request.
  code the fibonacci series in python using reccursion
  
model


    # noqa: E501
  """


  def __init__(self, response_to_request: str):
        if response_to_request is None:
            raise ValueError(
                "Missing the required property `response_to_request` when calling InstructionResponseV3.__init__"
            )

        self._to_python(response_to_request, 'response_to_request', True)

        self.additional_keys = None
        self.additional_types = None
        self.prompt_name = None
        self.instruction_name = None
        self.user_id = None
        self.status = None
        self.is_final = None
        self.log = None

    @property
    def response_to_request(self):
        """Gets the response_to_request of this InstructionResponseV3.

        The answer to the instruction.

        :return: The response_to_request of this Instructio

# Step 3 - Load dataset for finetuning

### 3.1 Download the Dataset from the github main branch

In [ ]:
!wget https://raw.githubusercontent.com/neomatrix369/learning-path-index/refs/heads/main/data/Learning_Pathway_Index.csv

--2024-10-18 22:28:38--  https://raw.githubusercontent.com/neomatrix369/learning-path-index/refs/heads/main/data/Learning_Pathway_Index.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 467720 (457K) [text/plain]
Saving to: ‘Learning_Pathway_Index.csv’

Learning_Pathway_In 100%[===================>] 456.76K  --.-KB/s    in 0.005s  

2024-10-18 22:28:38 (85.1 MB/s) - ‘Learning_Pathway_Index.csv’ saved [467720/467720]



### 3.2 Load the Dataset from current directory

In [ ]:
# Load your custom CSV file into a pandas DataFrame
csv_file_path = './Learning_Pathway_Index.csv'
df = pd.read_csv(csv_file_path)

# Convert the DataFrame to the Hugging Face Dataset format
dataset = Dataset.from_pandas(df)

In [ ]:
df = dataset.to_pandas()
df.head(10)

,Module_Code,Course_Learning_Material,Source,Course_Level,Type_Free_Paid,Module,Duration,Difficulty_Level,Keywords_Tags_Skills_Interests_Categories,Links
0,CLMML00,Introduction to Machine Learning,Google Developers,Beginner,Free,Introduction to Machine Learning,20.0 minutes,Easy,machine learning,https://developers.google.com/machine-learning...
1,CLMML00,Introduction to Machine Learning,Google Developers,Beginner,Free,What is Machine Learning,20.0 minutes,Easy,machine learning,https://developers.google.com/machine-learning...
2,CLMML00,Introduction to Machine Learning,Google Developers,Beginner,Free,Supervised Learning,20.0 minutes,Medium,supervised learning,https://developers.google.com/machine-learning...
3,CLMML00,Introduction to Machine Learning,Google Developers,Beginner,Free,Test your understanding,10.0 minutes,Easy,machine learning test,https://developers.google.com/machine-learning...
4,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Introduction to ML,3.0 minutes,Easy,machine learning,https://developers.google.com/machine-learning...
5,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Framing - Video Lecture,None,Medium,problem statement,https://developers.google.com/machine-learning...
6,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Framing - Key ML Terminology,15.0 minutes,Easy,ml terminologies,https://developers.google.com/machine-learning...
7,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Descending into ML - Video Lecture,None,Medium,machine learning,https://developers.google.com/machine-learning...
8,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Descending into ML - Linear Regression,None,Medium,"machine learning, linear regrression",https://developers.google.com/machine-learning...
9,CLMML01,Machine Learning Crash Course (Foundation),Google Developers,Intermediate,Free,Descending into ML - Training and Loss,None,Medium,"machine learning, training, loss",https://developers.google.com/machine-learning...


In [ ]:
def generate_prompt(data_point):
    instruction = data_point['Course_Learning_Material']
    input_text = data_point['Module']
    output = ""

    prefix_text = 'Below is an instruction that describes a task. Write a response that ' \
                  'appropriately completes the request.\n\n'
    if input_text:
        text = f"<start_of_turn>user {prefix_text} {instruction} here are the inputs {input_text} <end_of_turn>\n<start_of_turn>model{output} <end_of_turn>"
    else:
        text = f"<start_of_turn>user {prefix_text} {instruction} <end_of_turn>\n<start_of_turn>model{output} <end_of_turn>"
    return text

In [ ]:
# Apply the function to create prompts from the dataset
dataset = dataset.map(lambda x: {"prompt": generate_prompt(x)})

# Tokenize the dataset
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/1446 [00:00<?, ? examples/s]

Map:   0%|          | 0/1446 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['Module_Code', 'Course_Learning_Material', 'Source', 'Course_Level', 'Type_Free_Paid', 'Module', 'Duration', 'Difficulty_Level', 'Keywords_Tags_Skills_Interests_Categories', 'Links', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 1446
})

In [ ]:
# Split dataset into training and evaluation
# train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2)

# Split dataset into training and evaluation using HuggingFace's Dataset.train_test_split method
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']


In [ ]:
# Check the column names in the dataset
print(dataset.column_names)


['Module_Code', 'Course_Learning_Material', 'Source', 'Course_Level', 'Type_Free_Paid', 'Module', 'Duration', 'Difficulty_Level', 'Keywords_Tags_Skills_Interests_Categories', 'Links', 'prompt', 'input_ids', 'attention_mask']


In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/1156 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/290 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Convert logits to binary predictions if needed (for multi-label classification)
    if predictions.ndim > 2:
        predictions = predictions.argmax(axis=-1)  # if it's a multiclass case, get the class with the highest score
    else:
        predictions = (predictions > 0.5).astype(int)  # if it's a binary classification (logits), thresholding at 0.5

    # Now compute the metrics without unsupported 'average' arguments
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average='weighted', zero_division=0)
    recall = recall_score(labels, predictions, average='weighted', zero_division=0)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [ ]:
# # Split the initial dataset into train and test sets before adding prompts
# dataset_split = dataset.train_test_split(test_size=0.2)
# train_data = dataset_split["train"]
# test_data = dataset_split["test"]

# # Define and apply prompt generation for each split
# train_data = train_data.map(lambda x: {"prompt": generate_prompt(x)})
# test_data = test_data.map(lambda x: {"prompt": generate_prompt(x)})

# # Tokenize both splits
# train_data = train_data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
# test_data = test_data.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

In [ ]:
# print(test_data)

# Step 4 - Apply Lora  
Load a PeftModel and specify that we are going to use low-rank adapters (LoRA) using get_peft_model utility function and  the prepare_model_for_kbit_training method from PEFT.

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def find_all_linear_names(model):
  # if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  cls = bnb.nn.Linear4bit
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    # needed for 16-bit
    if 'lm_head' in lora_module_names:
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

In [ ]:
modules = find_all_linear_names(model)
print(modules)

['base_layer']


In [ ]:

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 432144384 | total: 9673850368 | Percentage: 4.4671%


# Step 5 - Run the training!

### 5.1 Fine-Tuning with qLora and Supervised Fine-Tuning

We're ready to fine-tune our model using qLora. For this tutorial, we'll use the `SFTTrainer` from the `trl` library for supervised fine-tuning. Ensure that you've installed the `trl` library as mentioned in the prerequisites.

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()


# trainer = SFTTrainer(
#     model=model,
#     train_dataset=train_data,
#     eval_dataset=test_data,
#     dataset_text_field="prompt",
#     peft_config=lora_config,
#     args=transformers.TrainingArguments(
#         per_device_train_batch_size=1,
#         gradient_accumulation_steps=4,
#         #warmup_steps=0.03,
#         max_steps=100,
#         learning_rate=2e-4,
#         logging_steps=1,
#         output_dir="outputs",
#         optim="paged_adamw_8bit",
#         save_strategy="epoch",
#     ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
# )


# Trainer initialization
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


## 5.2 Lets start training

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
# trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Epoch,Training Loss,Validation Loss


ValueError: multiclass-multioutput is not supported

 Share adapters on the 🤗 Hub

In [ ]:
new_model = "gemma2-lpi" #Name of the model you will be pushing to huggingface model hub

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

ValueError: Target modules {'base_layer'} not found in the base model. Please check the target modules and try again.

# Evaluation

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(f"Evaluation Results: {eval_results}")

# Plot loss curve (from logs)
loss_values = trainer.state.log_history
train_loss = [x['loss'] for x in loss_values if 'loss' in x]

plt.figure(figsize=(10, 5))
plt.plot(train_loss, label='Training Loss')
plt.title('Training Loss Curve')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

## 5.3 Publish to Hugging face

In [ ]:
# Push the model and tokenizer to the Hugging Face Model Hub
# merged_model.push_to_hub(new_model, use_temp_dir=False)
# tokenizer.push_to_hub(new_model, use_temp_dir=False)

## 5.4 Experimentation

In [ ]:
result = get_completion(query="what courses are Easy?", model=merged_model, tokenizer=tokenizer)

In [ ]:
print(result)

# Step 6 Web Application

In [ ]:
%%writefile app.py

import os
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from peft import PeftModel

# Set environment variable to avoid memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Define the offload directory
offload_dir = "/tmp"  # Change this to any writable directory on your system

# Define your base model ID
base_model_id = "merged_model"  # Replace with your actual model name

@st.cache_resource
def load_model():
    # Initialize model with empty weights
    with init_empty_weights():
        model = AutoModelForCausalLM.from_pretrained(base_model_id)

    # Load the model and offload layers to disk as needed
    model = load_checkpoint_and_dispatch(
        model, base_model_id, device_map="auto", offload_folder=offload_dir
    )

    # Load PEFT model
    new_model = "gemma2-lpi"
    peft_model = PeftModel.from_pretrained(model, new_model)

    tokenizer = AutoTokenizer.from_pretrained(new_model)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return peft_model, tokenizer

# Load model and tokenizer
model, tokenizer = load_model()

# Function to generate completions
def get_completion(query, model, tokenizer):
    inputs = tokenizer(query, return_tensors="pt", padding=True, truncation=True).to(model.device)
    outputs = model.generate(**inputs, max_length=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Streamlit App
st.title("Gemma Fine-Tuned Model Demo")
st.write("Enter a query and get a response from the fine-tuned model.")

query = st.text_input("Enter your query:")
if query:
    with st.spinner("Generating response..."):
        response = get_completion(query, model, tokenizer)
        st.write("Response:", response)


In [ ]:
!curl ipinfo.io

In [ ]:
# Run Streamlit in the background
!streamlit run app.py &>/dev/null &

# Use LocalTunnel to expose the Streamlit app
!lt --port 8501

In [ ]:
# Run the server
# !uvicorn app.py #app:app --host 0.0.0.0 --port 8000 &  # The `&` runs the server in the background

# Establish a tunnel to make it accessible over the internet
# !lt --port 8000 # --subdomain "your-custom-subdomain"
